# **Google Colab Instructions**

# **Image Filtering**
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque quis malesuada dui. Duis purus dui, imperdiet ultrices porta ut, tempus vel nisi. Proin vel urna sit amet mi sagittis iaculis quis vel magna. In laoreet eleifend enim quis dictum. Vivamus sed velit dictum, sagittis ipsum non, auctor erat. Nulla eget lobortis ligula, posuere faucibus erat. Nullam ac felis imperdiet libero viverra finibus sollicitudin consectetur libero. Quisque pharetra congue justo at varius. Nullam id quam nec sapien congue efficitur. Vivamus non scelerisque felis, id aliquam neque. Ut bibendum elementum diam, at efficitur nibh blandit ac. Vestibulum tincidunt tempor mattis.

**Purpose**
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque quis malesuada dui. Duis purus dui, imperdiet ultrices porta ut, tempus vel nisi.

**Necessary Requirements**
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque quis malesuada dui. Duis purus dui, imperdiet ultrices porta ut, tempus vel nisi.

**Expected Output**
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque quis malesuada dui. Duis purus dui, imperdiet ultrices porta ut, tempus vel nisi.

# **Python Setup**

**Dependencies**

In [ ]:
import cv2
import numpy as np            
import matplotlib.pyplot as plt

import os
from google.colab import drive
from google.colab import files

import ipywidgets as widgets
from pylab import rcParams

from ipywidgets import Output
from IPython.display import display
from ipywidgets import Button, HBox, VBox, Layout
from PIL import Image

# drive.mount('/content/drive')

**Function Setup**

In [ ]:
def display_one(a, title = "Original"):
    a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
    plt.imshow(a), plt.title(title)
    plt.show()

def filter(image, type, custom_kernel=0):
    if type in filters:
        kernel = filters.get(type)
        print(kernel)
    elif custom_kernel:
        kernel = custom_kernel
    elif type is "gaussian":
        return cv2.GaussianBlur(image, (35, 35), 0)
    elif type is "csharpen":
        ilambda = 0.1
        kernel = filters.get('laplacian') + (ilambda * filters.get('lowpass'))
        return cv2.filter2D(image, -1, kernel)
    else:
        return image
    return cv2.filter2D(image, -1, kernel)

def isolate_color(image, lower, upper):
    # Converts the image to the HSV colorspace
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Masks the image in the colorrange of [lower] to [upper]
    mask = cv2.inRange(hsv_image, lower, upper)

    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(image, image, mask=mask)

    return res

def file_upload():
  uploaded = files.upload()
  # for fn in uploaded.keys():
    # print('User uploaded file "{name}" with length {length} bytes'.format(
    #    name=fn, length=len(uploaded[fn])))
  return uploaded.keys()

def file_list_dropdown(Images):
  global initial_image
  file_name = Images
  initial_image = cv2.imread('/content/' + file_name)

def list(x):
  global filtered_image
  filtered_image = filter(initial_image, x)
  display_one(filtered_image, title=x)

'''
Displays the button for separating the HSV values
'''
def on_button_clicked(b):
  global hsv_separation_image
  # Activates after pressing the button
  with output:
    # Clears the cell if new input is provided
    output.clear_output(wait=True)
    # Grabs the upper and lower HSV values
    lower = np.array([lower_hue.value, lower_saturation.value, lower_value.value])
    upper = np.array([upper_hue.value, upper_saturation.value, upper_value.value])
    # Isolates the color of the image using the isolate_color method
    temp_img = cv2.cvtColor(isolate_color(initial_image, lower, upper), cv2.COLOR_BGR2RGB)
    hsv_separation_image = temp_img
    display(Image.fromarray(temp_img))
    # display_one(isolate_color(initial_image, lower, upper), title="Isolated")

def filter_clicked(b):
    with output_filter:
      filter = []

      counter = 0
      row = []
      for ibox in matrix:
        if counter % size == 0 and counter != 0:
          filter.append(row)
          row = []
        row.append(ibox.value)
        counter+=1
      filter.append(row)
      print('Adding Filter')
      print(filter)
      filter_list.append('Custom Filter' + str(custom_filter_count))
      filters['Custom Filter' + str(custom_filter_count)] = np.array(filter)

'''
Displays the two sliders for HSV separation
'''
def hsv_separation():
  print('Lower HSV range')
  display(lower_hue, lower_saturation, lower_value)
  print('Upper HSV range')
  display(upper_hue, upper_saturation, upper_value)

  lower_hsv = widgets.ColorPicker(
    concise=True,
    description='Pick a color',
    value='#0000',
    disabled=False
  )
  upper_hsv = widgets.ColorPicker(
    concise=True,
    description='Pick a color',
    value='#0000',
    disabled=False
  )

def filter_download(b):
  if os.path.exists(filter_name.value + '.jpg'):
    os.remove(filter_name.value + '.jpg')
  cv2.imwrite(filter_name.value + '.jpg', filtered_image)
  files.download('/content/' + filter_name.value + '.jpg') 

def hsv_download(b):
  if os.path.exists(hsv_name.value + '.jpg'):
    os.remove(hsv_name.value + '.jpg')
  bgr_hsv = cv2.cvtColor(hsv_separation_image, cv2.COLOR_RGB2BGR)
  cv2.imwrite(hsv_name.value +  '.jpg', bgr_hsv)
  files.download('/content/' + hsv_name.value + '.jpg') 

**Variable Initialization**

In [ ]:
rcParams['figure.figsize'] = 15, 15

# Stores the initial image
initial_image = None

# Stores the max/min HSV values for the sliders
lower_hue = widgets.IntSlider(value=0, max=179)
lower_saturation = widgets.IntSlider(value=0, max=255)
lower_value = widgets.IntSlider(value=0, max=255)

upper_hue = widgets.IntSlider(value=179, max=179)
upper_saturation = widgets.IntSlider(value=255, max=255)
upper_value = widgets.IntSlider(value=255, max=255)

# Global Variables to store the modified image
hsv_separation_image = np.array(1)
filtered_image = np.array(1)

button = widgets.Button(description="Isolate Color")
output = widgets.Output()

custom_filter_count = 0

# Sets up a button for downloading the filtered image
filter_button = widgets.Button(description="Download Filtered Image")
filter_output = widgets.Output()
filter_button.on_click(filter_download)

# Sets up a button for downloading the HSV separated image
hsv_button = widgets.Button(description="Download HSV Image")
hsv_output = widgets.Output()
hsv_button.on_click(hsv_download)

# Sets up Text Fields for entering file names
hsv_name = widgets.Text(
    value='hsv',
    description='HSV',
    disabled=False
)

filter_name = widgets.Text(
    value='filter',
    description='Filtered',
    disabled=False
)

**Design Your Own Filters Here!**

Example Filters listed below

In [ ]:
filters = {
    'none': np.array([ 1 ]),
    'emboss': np.array([[0, -1, -1],
                        [1, 0, -1],
                        [1, 1, 0]]),
           
    'highpass': np.array([[0.0, -1.0, 0.0],
                          [-1.0, 4.0, -1.0],
                          [0.0, -1.0, 0.0]]),
           
    'laplacian': np.array([[-1, 0, 1],
                           [-1, 0, 1],
                           [-1, 0, 1]]),
           
    'lowpass': np.array([[0, 0, 0],
                         [0, 5, 0],
                         [0, 0, 0]]),
           
    'sepia': np.array([[0.272, 0.534, 0.131],
                       [0.349, 0.686, 0.168],
                       [0.393, 0.769, 0.189]]),
           
    'sharpen': np.array([[-1, -1, -1],
                        [-1, 9, -1],
                        [-1, -1, -1]])
}
filter_list = filters.keys()
filter_list = [key for key in filter_list]
filter_list = filter_list + ['gaussian', 'csharpen']

In [ ]:
# matrix input box here
size_input = widgets.BoundedIntText(
    value=1,
    min=1,
    max=5,
    step=1,
    description='Matrix Size:',
    disabled=False
)
display(size_input)

BoundedIntText(value=1, description='Matrix Size:', max=5, min=1)

In [ ]:
matrix = []
size = size_input.value
newIntText = None
# print(size)
vertical = []
for item in range(0, size):
  horizontal = HBox()
  for f in range (0, size):
    newIntText = widgets.IntText()
    matrix.append(newIntText)
    horizontal.children += (newIntText,)
  vertical.append(horizontal)

display(VBox(vertical))

In [ ]:
add_filter = Button(description='Add Filter')
output_filter = widgets.Output()

add_filter.on_click(filter_clicked)

display(add_filter, output_filter)

Button(description='Add Filter', style=ButtonStyle())

Output()

# **Input**
Add images here! Be careful of having the same file names.

In [ ]:
files_keys = file_upload()
files_list = [key for key in files_keys]

Saving Screenshot (3).png to Screenshot (3) (1).png
Saving Screenshot (30).png to Screenshot (30).png


In [ ]:
widgets.interact(file_list_dropdown, Images=files_list)
hsv_separation()

interactive(children=(Dropdown(description='Images', options=('Screenshot (3).png', 'Screenshot (30).png'), va…

Lower HSV range


IntSlider(value=0, max=255)

IntSlider(value=0, max=255)

IntSlider(value=0, max=255)

Upper HSV range


IntSlider(value=255, max=255)

IntSlider(value=255, max=255)

IntSlider(value=255, max=255)

In [ ]:
button.on_click(on_button_clicked)
display(button, output)

Button(description='Isolate Color', style=ButtonStyle())

Output()

In [ ]:
widgets.interact(list, x=filter_list)

interactive(children=(Dropdown(description='x', options=('none', 'emboss', 'highpass', 'laplacian', 'lowpass',…

<function __main__.list>

# **Download**
Download the currently displayed image!

In [ ]:
print('Enter Filenames above Images (no extension required)')

display(hsv_name)
display(hsv_button, hsv_output)

display(filter_name)
display(filter_button, filter_output)

Enter Filenames above Images (no extension required)


Text(value='hsv', description='HSV')

Button(description='Download HSV Image', style=ButtonStyle())

Output()

Text(value='filter', description='Filtered')

Button(description='Download Filtered Image', style=ButtonStyle())

Output()

# **Exercise 1**
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque quis malesuada dui. Duis purus dui, imperdiet ultrices porta ut, tempus vel nisi. Proin vel urna sit amet mi sagittis iaculis quis vel magna. In laoreet eleifend enim quis dictum. Vivamus sed velit dictum, sagittis ipsum non, auctor erat. Nulla eget lobortis ligula, posuere faucibus erat. Nullam ac felis imperdiet libero viverra finibus sollicitudin consectetur libero. Quisque pharetra congue justo at varius. Nullam id quam nec sapien congue efficitur. Vivamus non scelerisque felis, id aliquam neque. Ut bibendum elementum diam, at efficitur nibh blandit ac. Vestibulum tincidunt tempor mattis.

# **Exercise 2**
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque quis malesuada dui. Duis purus dui, imperdiet ultrices porta ut, tempus vel nisi. Proin vel urna sit amet mi sagittis iaculis quis vel magna. In laoreet eleifend enim quis dictum. Vivamus sed velit dictum, sagittis ipsum non, auctor erat. Nulla eget lobortis ligula, posuere faucibus erat. Nullam ac felis imperdiet libero viverra finibus sollicitudin consectetur libero. Quisque pharetra congue justo at varius. Nullam id quam nec sapien congue efficitur. Vivamus non scelerisque felis, id aliquam neque. Ut bibendum elementum diam, at efficitur nibh blandit ac. Vestibulum tincidunt tempor mattis.